# Natural Language Processing + Decision Science

👩🏻‍🏫 In this challenge, we will combine:
* 🗣 Natural Language Processing 
* 📊 Decision Science 

🎯 The goal is to understand the bad reviews of the products and the sellers on Olist

In [3]:
# Data Manipulation
import numpy as np
import pandas as pd
pd.set_option("max_columns",None)

# Olist packages
from olist.data import Olist
from olist.review import Review
from olist.order import Order
from olist.product_updated import Product
from olist.seller_updated import Seller

# Machine Learning
from sklearn.pipeline import make_pipeline

# Language Processing
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 
from nltk.stem import WordNetLemmatizer
import string
import unidecode

# Vectorizers and NLP Models
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

🕵🏻‍♂️ Imagine that the CEO of Olist [Tiago Dalvi](https://www.linkedin.com/in/tiagodalvi/) ask you to read and understand the reviews.

- What did customers say about their order if they rated it 1? 2? 3?
- What are the most frequent bad reviews about...
    - the worst rated products ?
    - the wort sellers ?

## (0) Setup

👩🏻‍🏫 If you followed the `Decision Science` module, you already have the `olist` package installed and importable. *You can skip this section* and move to the **Data collection** section.

### (0.1) Import `olist` package`

Download a fresh version of the `olist` package:

```bash
mkdir ~/code/lewagon
cd ~/code/lewagon
git clone git@github.com:lewagon/olist.git
cd olist
git fetch
git checkout full-package
```

### (0.2) Download the datasets

- Download the datasets from Kaggle https://www.kaggle.com/olistbr/brazilian-ecommerce
- Unzip them into the `/data/csv` directory of the `olist` package:

```bash
.
├── README.md
├── data
│   └── csv
│       ├── olist_customers_dataset.csv
│       ├── olist_geolocation_dataset.csv
│       ├── olist_order_items_dataset.csv
│       ├── olist_order_payments_dataset.csv
│       ├── olist_order_reviews_dataset.csv
│       ├── olist_orders_dataset.csv
│       ├── olist_products_dataset.csv
│       ├── olist_sellers_dataset.csv
│       └── product_category_name_translation.csv
├── notebooks
├── olist
│   ├── README.md
│   ├── __init__.py
│   ├── data.py
│   ├── order.py
│   ├── product.py
│   ├── product_updated.py
│   ├── review.py
│   ├── seller.py
│   ├── seller_updated.py
│   └── utils.py
├── requirements.txt
└── setup.p
```

### (0.3) Install the `olist` package

```bash
pip install -e .
```

⚠️ Restart the kernel.

## (1) Data Collection and Preparation

👉 The way we designed the `reviews` DataFrame will help us focus on the `product categories`.

In [4]:
reviews = Review().get_training_data()
reviews.head()

,review_id,length_review,review_score,order_id,product_category_name
0,7bc2406110b926393aa56f80a40eba40,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer
1,80e641a11e56f04c1ad469d5645fdfde,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios
2,228ce5500dc1d8e020d8d1322874b6f0,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer


In [6]:
everyone = ['Belu', 'Leah', 'Roselyn', 'Jiaying', 'Andy']

In [9]:
import random
random.choice(everyone)

'Roselyn'

👉 We can retrieve the reviews from `order_reviews` within the Olist class.

In [10]:
data = Olist().get_data()

In [11]:
reviews_data = data['order_reviews']
reviews_data.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


🪤 We need to collect some information about the orders because:
- Back then, customers could review an order even before receiving it
- They could also rate an order that they hadn't received

In [12]:
orders = data['orders']
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


💥 Let's merge these three datasets and apply some operations

In [16]:
reviews.shape

(98657, 5)

In [17]:
reviews_data.shape

(99224, 7)

In [18]:
orders.shape

(99441, 8)

In [27]:
merged_df=reviews_data.merge(reviews,on='review_id').drop(columns='order_id_x')\
.rename(columns={'order_id_y':'order_id'})\
.merge(orders,on='order_id')

In [28]:
merged_df.shape

(101937, 17)

In [30]:
merged_df.head()

,review_id,review_score_x,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,length_review,review_score_y,order_id,product_category_name,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,80e641a11e56f04c1ad469d5645fdfde,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,228ce5500dc1d8e020d8d1322874b6f0,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,e64fb393e7b32834bb789ff8bb30750e,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,f7c4243c7fe1938f181bec41a392bdeb,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00


In [35]:
merged_df=merged_df.drop_duplicates()

In [36]:
merged_df.shape

(97077, 17)

In [37]:
merged_df.dtypes

review_id                        object
review_score_x                    int64
review_comment_title             object
review_comment_message           object
review_creation_date             object
review_answer_timestamp          object
length_review                     int64
review_score_y                    int64
order_id                         object
product_category_name            object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [40]:
merged_df[['order_delivered_carrier_date','order_delivered_customer_date',\
                         'order_estimated_delivery_date']]

,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02 00:00:00
1,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14 00:00:00
2,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09 00:00:00
3,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10 00:00:00
4,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09 00:00:00
...,...,...,...
101932,2018-06-29 14:25:00,2018-07-06 13:22:37,2018-07-20 00:00:00
101933,2017-12-07 19:17:29,2017-12-08 17:19:00,2017-12-20 00:00:00
101934,2018-03-19 23:51:29,2018-03-21 17:44:08,2018-04-06 00:00:00
101935,2018-06-25 12:58:00,2018-06-30 12:57:51,2018-07-16 00:00:00


In [45]:
merged_df['order_delivered_carrier_date']=pd.to_datetime(merged_df['order_delivered_carrier_date'])
merged_df['order_delivered_customer_date']=pd.to_datetime(merged_df['order_delivered_customer_date'])
merged_df['order_estimated_delivery_date']=pd.to_datetime(merged_df['order_estimated_delivery_date'])
merged_df['review_creation_date']=pd.to_datetime(merged_df['review_creation_date'])

/Users/jinru/Projects/le_wagon_lectures/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/jinru/Projects/le_wagon_lectures/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/jinru/Projects/le_wagon_lectures/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a 

In [46]:
merged_df.dtypes

review_id                                object
review_score_x                            int64
review_comment_title                     object
review_comment_message                   object
review_creation_date             datetime64[ns]
review_answer_timestamp                  object
length_review                             int64
review_score_y                            int64
order_id                                 object
product_category_name                    object
customer_id                              object
order_status                             object
order_purchase_timestamp                 object
order_approved_at                        object
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [49]:
merged_df=merged_df.drop(columns='review_score_x')\
.rename(columns={'review_score_y':'review_score'})

In [50]:
merged_df.dtypes

review_id                                object
review_comment_title                     object
review_comment_message                   object
review_creation_date             datetime64[ns]
review_answer_timestamp                  object
length_review                             int64
review_score                              int64
order_id                                 object
product_category_name                    object
customer_id                              object
order_status                             object
order_purchase_timestamp                 object
order_approved_at                        object
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

🚓 Remove the reviews that were written even before receiving the order.

In [52]:
merged_df.head()

,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,length_review,review_score,order_id,product_category_name,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,NaN,NaN,2018-01-18,2018-01-18 21:46:59,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02
1,80e641a11e56f04c1ad469d5645fdfde,NaN,NaN,2018-03-10,2018-03-11 03:05:13,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14
2,228ce5500dc1d8e020d8d1322874b6f0,NaN,NaN,2018-02-17,2018-02-18 14:36:24,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09
3,e64fb393e7b32834bb789ff8bb30750e,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10
4,f7c4243c7fe1938f181bec41a392bdeb,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09


In [54]:
merged_df=merged_df[merged_df['review_creation_date']>merged_df['order_delivered_customer_date']]

In [55]:
merged_df.shape

(86832, 16)

🚓 Remove the reviews for undelivered orders.

In [56]:
merged_df=merged_df[merged_df['order_status']=='delivered']

In [57]:
merged_df.shape

(86827, 16)

✍️ As some people fill in only the title or the body of a review, it may be a good idea to aggregate them together.



<details>
    <summary>💡<i>Hint</i></summary>

* Drop rows with either a missing title or a missing comment
* Refer to the documentation [pandas.DataFrame.dropna()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.dropna.html) and have a look at the `how` argument...

In [58]:
merged_df.head()

,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,length_review,review_score,order_id,product_category_name,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,7bc2406110b926393aa56f80a40eba40,NaN,NaN,2018-01-18,2018-01-18 21:46:59,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02
1,80e641a11e56f04c1ad469d5645fdfde,NaN,NaN,2018-03-10,2018-03-11 03:05:13,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14
2,228ce5500dc1d8e020d8d1322874b6f0,NaN,NaN,2018-02-17,2018-02-18 14:36:24,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09
3,e64fb393e7b32834bb789ff8bb30750e,NaN,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10
4,f7c4243c7fe1938f181bec41a392bdeb,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09


In [65]:
#turn nan to empty string
merged_df[['review_comment_title','review_comment_message']]=\
merged_df[['review_comment_title','review_comment_message']].fillna('')

In [70]:
#aggregate
merged_df['aggregated_review']=\
merged_df['review_comment_title']+' '+merged_df['review_comment_message']
merged_df.head()

,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,length_review,review_score,order_id,product_category_name,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,aggregated_review
0,7bc2406110b926393aa56f80a40eba40,,,2018-01-18,2018-01-18 21:46:59,0,4,73fc7af87114b39712e6da79b0a377eb,esporte_lazer,41dcb106f807e993532d446263290104,delivered,2018-01-11 15:30:49,2018-01-11 15:47:59,2018-01-12 21:57:22,2018-01-17 18:42:41,2018-02-02,
1,80e641a11e56f04c1ad469d5645fdfde,,,2018-03-10,2018-03-11 03:05:13,0,5,a548910a1c6147796b98fdf73dbeba33,informatica_acessorios,8a2e7ef9053dea531e4dc76bd6d853e6,delivered,2018-02-28 12:25:19,2018-02-28 12:48:39,2018-03-02 19:08:15,2018-03-09 23:17:20,2018-03-14,
2,228ce5500dc1d8e020d8d1322874b6f0,,,2018-02-17,2018-02-18 14:36:24,0,5,f9e4b658b201a9f2ecdecbb34bed034b,informatica_acessorios,e226dfed6544df5b7b87a48208690feb,delivered,2018-02-03 09:56:22,2018-02-03 10:33:41,2018-02-06 16:18:28,2018-02-16 17:28:48,2018-03-09,
3,e64fb393e7b32834bb789ff8bb30750e,,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09,Parabéns lojas lannister adorei comprar pela ...


In [71]:
#drop
merged_df=merged_df[merged_df['aggregated_review']!=' ']

In [75]:
merged_df.head()

,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,length_review,review_score,order_id,product_category_name,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,aggregated_review
3,e64fb393e7b32834bb789ff8bb30750e,,Recebi bem antes do prazo estipulado.,2017-04-21,2017-04-21 22:02:06,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,de6dff97e5f1ba84a3cd9a3bc97df5f6,delivered,2017-04-09 17:41:13,2017-04-09 17:55:19,2017-04-10 14:24:47,2017-04-20 09:08:35,2017-05-10,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,,Parabéns lojas lannister adorei comprar pela I...,2018-03-01,2018-03-02 10:26:53,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,5986b333ca0d44534a156a52a8e33a83,delivered,2018-02-10 10:59:03,2018-02-10 15:48:21,2018-02-15 19:36:14,2018-02-28 16:33:35,2018-03-09,Parabéns lojas lannister adorei comprar pela ...
9,8670d52e15e00043ae7de4c01cc2fe06,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22,2018-05-23 16:45:47,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,a5224bdc7685fd39cd7a23404415493d,delivered,2018-05-14 10:29:02,2018-05-15 10:37:47,2018-05-15 13:29:00,2018-05-21 17:52:12,2018-06-06,recomendo aparelho eficiente. no site a marca ...
12,4b49719c8a200003f700d3d986ea1a19,,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16,2018-02-20 10:52:22,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,8ebbeb09e94d1e789142983f73faa9cd,delivered,2018-02-12 16:07:34,2018-02-12 16:27:39,2018-02-14 20:04:16,2018-02-15 19:16:54,2018-02-26,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23,2018-05-24 03:00:01,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,659ded3e9b43aaf51cf9586d03033b46,delivered,2018-05-18 18:20:45,2018-05-18 18:35:28,2018-05-19 09:27:00,2018-05-22 14:58:47,2018-06-07,"Super recomendo Vendedor confiável, produto ok..."


In [74]:
merged_df.shape

(35729, 17)

In [77]:
def preprocessing(sentence):
    
    import numpy as np
    def remove_punctuation(text):
        import string
        for char in string.punctuation:
            text = text.replace(char, "")
        return text
    
    def lower_case(text):
        return text.lower()
    
    def remove_numbers(text):
        return "".join(char for char in text if not char.isdigit())
    
    def remove_stopwords(text):
        import string
        from nltk.corpus import stopwords 
        from nltk.tokenize import word_tokenize

        stop_words = list(stopwords.words('portuguese'))
        for word in stop_words:
            for char in string.punctuation:
                word = word.replace(char, "")
            if word not in stop_words:
                stop_words.append(word)

        text_token = word_tokenize(text)
        non_stop_tokens = [w for w in text_token if not w in stop_words]
        return ' '.join(w for w in non_stop_tokens)
    
    def lemmatize(text):
        import nltk
        from nltk.tokenize import word_tokenize

        text_token = word_tokenize(text)


        verb_lemmatized = [nltk.WordNetLemmatizer().lemmatize(word, pos = "v")  # v --> verbs
                  for word in text_token]


        noun_lemmatized = [nltk.WordNetLemmatizer().lemmatize(word, pos = "n")  # n --> nouns
                  for word in verb_lemmatized]

        return ' '.join(w for w in noun_lemmatized)
    
    return lemmatize(remove_stopwords(remove_numbers(lower_case(remove_punctuation(str(sentence)))))) if sentence != np.nan else sentence

🕵🏻 Let's focus on some columns of interest:

In [78]:
# Focus on certain columns
columns_of_interest = ['review_id',
                       'length_review',
                       'review_score',
                       'order_id',
                       'product_category_name',
                       'aggregated_review'
                      ]
df = merged_df[columns_of_interest]

In [79]:
df.head()

,review_id,length_review,review_score,order_id,product_category_name,aggregated_review
3,e64fb393e7b32834bb789ff8bb30750e,37,5,658677c97b385a9be170737859d3511b,ferramentas_jardim,Recebi bem antes do prazo estipulado.
4,f7c4243c7fe1938f181bec41a392bdeb,100,5,8e6bfb81e283fa7e4f11123a3fb894f1,esporte_lazer,Parabéns lojas lannister adorei comprar pela ...
9,8670d52e15e00043ae7de4c01cc2fe06,174,4,b9bf720beb4ab3728760088589c62129,eletroportateis,recomendo aparelho eficiente. no site a marca ...
12,4b49719c8a200003f700d3d986ea1a19,45,4,9d6f15f95d01e79bd1349cc208361f09,beleza_saude,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
15,3948b09f7c818e2d86c9a546758b2335,56,5,e51478e7e277a83743b6f9991dbfa3fb,informatica_acessorios,"Super recomendo Vendedor confiável, produto ok..."


## (2) Text Cleaning

🧹 Create a function `cleaning(sentence)` and apply it to the reviews.

In [80]:
df["aggregated_review"] = df.aggregated_review.apply(preprocessing)
df["aggregated_review"].head()

/Users/jinru/Projects/le_wagon_lectures/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


3                      recebi bem ante prazo estipulado
4     parabéns lojas lannister adorei comprar intern...
9     recomendo aparelho eficiente site marca aparel...
12                      pouco travandopelo valor ta boa
15    super recomendo vendedor confiável produto ok ...
Name: aggregated_review, dtype: object

## (3) Analysis of bad reviews

### (3.1) Dataset with low review scores

😱 What is the proportion of reviews with a rating between 1 and 3 ? 

In [81]:
def bad_score(score):
    return 0 if score > 3 else 1
df["bad_score"] = df.review_score.apply(bad_score)
df[["bad_score", "review_score"]].sample(5)

/Users/jinru/Projects/le_wagon_lectures/lewagon/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,bad_score,review_score
4236,1,2
43137,0,4
28137,1,2
34261,1,1
7341,0,5


🕵🏻‍♂️ Let's focus on these reviews...

In [ ]:
# YOUR CODE HERE

### (3.2) Vectorizing

🔡 ➡️ 🔢 Vectorize your texts. 

* Make sure to take into accounts bigrams.
* Set a `max_df` to $0.75$ to remove too frequent words
* Spoiler alert: you will end up with $20k+$ words... let's keep only `max_features` = $5000$ for this challenge.

In [99]:
from sklearn.feature_extraction.text import TfidfVectorizer

X = df[:10000].aggregated_review
y = df[:10000].bad_score

tfid = TfidfVectorizer(max_df=.75, ngram_range=(1,2), max_features=5000)
X = tfid.fit_transform(X)

### (3.3) LDA

🕵🏻‍♂️ Fit an LDA:
- Choose `n_components = 3`
- Show the Document Mixture of Topics  with *.transform()*
- Show the Topic Mixture with *.components_*

In [100]:
from sklearn.decomposition import LatentDirichletAllocation

lda_model = LatentDirichletAllocation(n_components=3, max_iter = 10)

lda_model.fit(X)

LatentDirichletAllocation(n_components=3)

#### Document Mixture (of Topics)

👉 Let's report the most important topic for each review

In [103]:
lda_model.transform(X)

array([[0.08922157, 0.82042262, 0.09035581],
       [0.47656671, 0.14360802, 0.37982527],
       [0.1851163 , 0.11700892, 0.69787479],
       ...,
       [0.11211969, 0.10576169, 0.78211862],
       [0.47913377, 0.43517543, 0.0856908 ],
       [0.37075926, 0.54272768, 0.08651306]])

#### Topic Mixture (of Words)

In [110]:
topic_word_mixture = pd.DataFrame(lda_model.components_, 
                                 columns = tfid.get_feature_names_out())

In [111]:
topic_word_mixture

abaixo    aberta    aberto    abraço   abraços      abre      abri  \
0  0.353098  0.522877  1.114470  1.708566  0.339243  0.385633  1.989218   
1  0.378032  0.341479  0.338597  0.336214  0.774852  0.640184  0.335616   
2  3.297053  3.954550  2.309518  0.358703  1.978898  0.946055  4.924569   

   abri caixa  abri embalagem  abri reclamação   abrindo     abrir   absurdo  \
0    1.644473        1.126555         0.361381  1.324220  3.651904  2.080954   
1    0.335591        0.336873         0.334384  0.336557  0.342466  0.754101   
2    0.371777        0.897489         1.954522  0.335876  2.100056  0.689685   

      acaba   acabado  acabamento  acabamento perfeito  acabamento ruim  \
0  0.941878  0.348438    4.428851             1.425119         0.349308   
1  0.381699  0.345456    4.468544             0.336532         1.976581   
2  0.775701  5.513681    8.724347             1.761369         0.364276   

     acabei  acabei recebendo    acabou   aceitei    acende    acesso  \
0  3.414142          1.248289  0.416896  0.344550  0.817064  0.348895   
1  0.345066          0.347405  2.038420  0.336036  0.654098  0.580615   
2  0.734785          0.350596  0.955680  1.244890  0.383710  1.258255   

   acessível  acessórios   achando      achei  achei bem  achei demorou  \
0   0.336353    0.335987  1.335518   9.580237   0.339381       0.337021   
1   2.300459    0.335980  0.335052   7.446396   0.342175       0.678495   
2   0.353073    3.771588  0.988469  16.772897   1.501886       1.542384   

   achei frete  achei lindo  achei melhor  achei pouco  achei produto  \
0     0.349144     2.235832      1.362191     0.875337       2.117413   
1     2.038707     0.351858      0.334733     0.444730       1.399130   
2     0.350452     0.334915      0.350672     2.894018       0.516565   

   achei qualidade  achei ruim      acho  acho deveria  acho deveriam  \
0         1.507247    0.349929  6.766782      1.152429       1.850229   
1         0.351115    0.342839  5.120095      1.377942       0.350507   
2         0.335201    1.500395  2.996849      0.459341       0.348696   

   acho veio     acima  acompanha  acompanhamento  acompanhar  acondicionado  \
0   1.169481  1.869185   0.475777        1.800351    0.634407       0.340308   
1   0.404953  1.007105   0.341843        0.486514    0.345170       0.387668   
2   0.582500  0.353880   1.891096        0.394959    1.757923       2.639445   

   acontece  aconteceu  acordado     acordo  acordo anunciado  acordo anuncio  \
0  1.683295   4.395841  0.367377   0.352779          0.336399        0.337340   
1  0.363689   0.389196  2.309949  14.537075          2.243961        0.370021   
2  0.939198   0.357013  0.334978   5.672066          0.347889        1.544847   

   acordo descrição  acordo foto  acredito  acredito conta     acusa  \
0          0.346741     0.341959  2.527449        0.400912  1.122473   
1          2.445654     0.339835  3.409309        1.260928  0.336380   
2          0.539491     1.693656  0.401350        0.334642  0.778233   

   adaptador  adequada  adequado   adesiva   adesivo  adesivos  adoramos  \
0   0.384563  0.361437  2.090848  0.412528  3.685052  1.771153  0.337591   
1   0.335949  1.205310  0.335593  0.335126  0.370893  0.703608  2.512980   
2   2.819049  0.843461  0.342634  1.097009  0.533269  0.369137  0.347372   

      adorei  adorei chegou  adorei compra  adorei lindo  adorei mercadoria  \
0  79.492826       2.463380       0.463864      4.609899           1.580839   
1   0.382659       0.404119       0.342760      0.353304           0.340801   
2   0.361592       0.335258       1.731867      0.339613           0.334391   

   adorei produto  adorei recomendo     adoro  adoro comprar     adorou  \
0       17.896281          3.395693  0.362947       0.380515  12.595607   
1        0.369656          0.341909  0.347340       0.345355   0.394220   
2        0.347437          0.338360  9.323781       4.374529   0.346204   

    adquiri  adquiri produto  adquirido  adq

#### Topics

🎁 We provided you with some functions:
* `topic_word` returns the top words with their weights for one topic
* `print_topics` prints the different topics found by the LDA with their topwords

In [115]:
def topic_word(vectorizer, model, topic, topwords, with_weights = True):
    topwords_indexes = topic.argsort()[:-topwords - 1:-1]
    if with_weights == True:
        topwords = [(vectorizer.get_feature_names_out()[i], round(topic[i],2)) for i in topwords_indexes]
    if with_weights == False:
        topwords = [vectorizer.get_feature_names_out()[i] for i in topwords_indexes]
    return topwords

In [121]:
def print_topics(vectorizer, model, topwords):
    topics = []
    for idx, topic in enumerate(model.components_):
        print("-"*20)
        print("Topic %d:" % (idx))
        print(type(topic_word(vectorizer, model, topic, topwords)))
        print(topic_word(vectorizer, model, topic, topwords))
        topics.append(topic_word(vectorizer, model, topic, topwords))
    return pd.DataFrame(topics)
        

🕵🏻‍♂️ Print the topics with their topwords:

In [122]:
df = print_topics(tfid, lda_model, 10)

--------------------
Topic 0:
<class 'list'>
[('ótimo', 233.53), ('excelente', 169.11), ('gostei', 147.84), ('otimo', 147.31), ('produto', 145.41), ('boa', 143.84), ('adorei', 79.49), ('ótimo produto', 76.69), ('loja', 71.76), ('comprei', 66.11)]
--------------------
Topic 1:
<class 'list'>
[('recomendo', 364.29), ('prazo', 285.02), ('entrega', 222.53), ('ante', 207.3), ('produto', 205.33), ('ante prazo', 193.08), ('super', 155.91), ('entregue', 135.03), ('rápida', 113.82), ('chegou', 110.52)]
--------------------
Topic 2:
<class 'list'>
[('bom', 457.2), ('produto', 147.17), ('ótima', 102.22), ('veio', 88.44), ('bom produto', 74.88), ('recebi', 73.74), ('nota', 70.88), ('parabéns', 57.86), ('chegou', 57.53), ('compra', 51.91)]


🇧🇷 A bit of Brazilian Portuguese words here:
- _cadeiras = chairs_
- _producto = product_
- _bom = good_
- _comprei = bought_
- _veio = came_
- _duas = two_
- _nao = not_
- _entregue = delivered_
- _pecas = part_
- _ainda = yet_
- _recebi = received_

👉 Show the top words associated to a topic

In [133]:
topic_word_mixture['abaixo']

0    0.353098
1    0.378032
2    3.297053
Name: abaixo, dtype: float64

In [118]:
df["most_important_words"] = df["most_important_topic"].apply(lambda i: topic_word_mixture[i])

KeyError: 'most_important_topic'

In [117]:
df[["review_id", 
        "review_score", 
        "product_category_name",
        "full_review_cleaned",
        "most_important_topic",
        "most_important_words"]
      ].head(3)

KeyError: "['most_important_topic', 'full_review_cleaned', 'most_important_words'] not in index"

## (3.4) Pipeline Tf-Idf and LDA

In [ ]:
from sklearn import set_config
set_config("diagram")

🔨 Create a Pipeline that chains your previous Vectorizer and the LDA.

Fit it on the cleaned texts.

In [ ]:
# YOUR CODE HERE

💡 If you try to access the components with `pipeline.components_`, it will NOT work because Pipeline doesn't have a `components_`. However, you can use `pipeline._final_estimator` to access the LDA. And from this, you can access the topics!

In [ ]:
pipeline._final_estimator

In [ ]:
pipeline._final_estimator.components_

**Document Mixture** with the Pipeline:

In [ ]:
# YOUR CODE HERE

**Topic Mixture** with the Pipeline:

In [ ]:
# YOUR CODE HERE

## (4) 🎁 Product Categories

### (4.1) Groupby product categories

📈 Group the dataset by `product_category_name` and inspect their performances

In [ ]:
# Product categories by performance - look at the count, mean, median and std
product_categories = df.groupby(by = 'product_category_name').agg({
        'review_score': ["count", "mean", "median", "std"]
    })

# Removing products which were sold less than a certain times for the analysis
cutoff = 50
product_categories = product_categories[product_categories[("review_score", "count")] > cutoff]

# Sorting the product categories by performance
product_categories = product_categories.sort_values(by = [('review_score', 'mean'), 
                                                          ('review_score', 'std')], 
                                                    ascending = [False, True])
product_categories

### (4.2) Worst product categories

👎 Store the five worst categories in terms of *average review score* into a variable called `worst_products`

In [ ]:
worst_products = product_categories.tail(5).sort_values(by = [("review_score", "count")],
                                                       ascending = False)
worst_products

👇 Create a `worst_products_review` DataFrame which contains the `worst_products` only.

In [ ]:
worst_products_reviews = df[df.product_category_name.isin(worst_products.index)]
worst_products_reviews[["review_id", 
                        "review_score", 
                        "product_category_name",
                        "full_review_cleaned",
                        "most_important_topic",
                        "most_important_words"]
      ]

### (4.3). Topics for the worst products

❓ What are the topics for the worst products ❓

In [ ]:
worst_products_reviews["most_important_topic"].value_counts()

In [ ]:
bad_frequency = list(worst_products_reviews["most_important_topic"].value_counts().index)
bad_frequency

In [ ]:
[topic_word_mixture[i] for i in bad_frequency]

## (5) 🎁 Sellers...

* What kind of products were sold by the worst sellers ?
* What are the main reviews for the worst sellers ?

### (5.1) Worst Sellers

In [ ]:
sellers = Seller().get_training_data()
sellers.columns

👇 Select the 10 worst sellers and store them into a variable called `worst_sellers`

In [ ]:
worst_sellers = sellers[["seller_id", "review_score", "profits"]].sort_values(
    by = "profits", 
    ascending = True).head(10)
worst_sellers

### (5.2) Products sold by the worst sellers

In [ ]:
products = Product().get_training_data() [["product_id", "category"]]
products

❓ What are the types of products sold by the worst sellers ❓

In [ ]:
sellers_product_category = data["order_items"].merge(products, 
                                             on = "product_id", how = "left")[["seller_id", "category"]]

sellers_product_category

In [ ]:
sellers_product_category.groupby("seller_id").count()

### (5.3) Categories and topics for the worst sellers

🎁 Here are some useful functions:
- `focus_seller(seller_id)` to show the product categories sold by a seller
- `bad_reviews_seller` to show to topwords of the most frequent topic for one seller

In [ ]:
def focus_seller(seller_id):
    return sellers_product_category[sellers_product_category.seller_id == seller_id].value_counts()

In [ ]:
bad_reviews_sellers = df.merge(data["order_items"])
bad_reviews_sellers.head(3)

In [ ]:
def bad_reviews_seller(bad_reviews_sellers, seller_id):
    mask = (bad_reviews_sellers.seller_id == seller_id)
    temp = bad_reviews_sellers[mask]
    most_frequent_topic_seller = list(temp.most_important_topic.value_counts().head(1).index)[0]
    return topic_word_mixture[most_frequent_topic_seller]

❓For each of these worst sellers, show the most frequent product categories and words ❓

In [ ]:
for worst_seller in worst_sellers["seller_id"]:
    print("-"*50)
    print(f"Focusing on the seller #{worst_seller}...")
    print(focus_seller(worst_seller))
    print(bad_reviews_seller(bad_reviews_sellers, worst_seller))
    

🏁 Congratulations. You've learned some basics of NLP (Preprocessing + Vectorizing + NB/LDA) and we combined this new "expertise" with Decision Science

💾 Don't forget to `git add / commit / push`